# アンサンブル検討用nb（XGboost、LightGBM、Catboost）

## Mount＆modjule import

In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
# 目的：スペインの電力価格を予測
# 目的変数：スペインの電力価格(actual_price)
# 評価指標：RME

import numpy as np
import pandas as pd
import os
import pickle
import gc
# 分布確認
!pip install ydata-profiling
from ydata_profiling import ProfileReport
# 可視化
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder,OneHotEncoder
# モデリング
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from xgboost import XGBClassifier
import lightgbm as lgb
import xgboost as xgb
!pip install -U xgboost -q
!pip install -q catboost
from catboost import CatBoostRegressor, Pool

# 日本語表記
!pip install japanize-matplotlib
import japanize_matplotlib
%matplotlib inline
# パラメータ最適化
!pip install optuna
import optuna

# 評価指標
from sklearn.metrics import mean_squared_error

In [84]:
file_path = '/content/drive/MyDrive/Colab Notebooks/signate/smbc/'

df_train = pd.read_csv(file_path + 'train.csv')
df_test = pd.read_csv(file_path + 'test.csv')

## 前処理

In [85]:
# 時刻をdatetimeに変換
df_train["time"] = pd.to_datetime(df_train["time"],utc=True)
df_test["time"] = pd.to_datetime(df_test["time"],utc=True)

# 年情報のカラム作成（クロスバリデーション用）
df_train["year"] = df_train["time"].dt.year
df_test["year"] = df_test["time"].dt.year

# ケルビンを摂氏に変換  - 273.15
cities = ["valencia","madrid","bilbao","barcelona","seville"]
temp_cols = [f"{c}_{t}" for c in cities for t in ["temp", "temp_min", "temp_max"]]

for df in (df_train, df_test):          # ★ 2 つとも回す
    for col in temp_cols:
        df[col] = df[col] - 273.15

print("train temp range :", df_train["seville_temp_min"].min(), df_train["seville_temp_min"].max())
print("test  temp range :", df_test ["seville_temp_min"].min(), df_test ["seville_temp_min"].max())


train temp range : -4.0 42.0
test  temp range : -3.0 42.0


## 特徴量エンジニアリング

In [86]:
# 季節性（spring:1,summer:2,autumn:3,winter:4）
def get_season(month):
    if month in [3, 4, 5]:
        return 1
    elif month in [6, 7, 8]:
        return 2
    elif month in [9, 10, 11]:
        return 3
    else:
        return 4

# 月を取得 → 季節に変換
df_train["month"] = df_train["time"].dt.month
df_train["season"] = df_train["month"].apply(get_season)

# テストデータにも
df_test["month"] = df_test["time"].dt.month
df_test["season"] = df_test["month"].apply(get_season)

In [87]:
# 時間帯（morning:1,afternoon:2,evening:3,night:4）

def get_time_of_day(hour):
    if 5 <= hour < 11:
        return 1
    elif 11 <= hour < 17:
        return 2
    elif 17 <= hour < 21:
        return 3
    else:
        return 4

df_train["hour"] = df_train["time"].dt.hour
df_train["time_of_day"] = df_train["hour"].apply(get_time_of_day)

# テストデータにも
df_test["hour"] = df_test["time"].dt.hour
df_test["time_of_day"] = df_test["hour"].apply(get_time_of_day)

In [88]:
# 時間帯：夜とそれ以外（night:1,others:0）

def get_time_of_day_only_night(hour):
    if 5 <= hour < 21:
        return 0
    else:
        return 1

df_train["hour"] = df_train["time"].dt.hour
df_train["time_of_day_only_night"] = df_train["hour"].apply(get_time_of_day)

# テストデータにも
df_test["hour"] = df_test["time"].dt.hour
df_test["time_of_day_only_night"] = df_test["hour"].apply(get_time_of_day)

In [89]:
# 週末フラグ

def is_week_end(weekday):
    if weekday in [0,1,2,3,4]:
      return 0
    elif weekday in [5,6]:
      return 1
    else:
      return 2

df_train["weekday"] = df_train["time"].dt.weekday
df_train["is_weekend"] = df_train["weekday"].apply(is_week_end)

# テストデータにも
df_test["weekday"] = df_test["time"].dt.weekday
df_test["is_weekend"] = df_test["weekday"].apply(is_week_end)

In [90]:
# 発電量の和
generation_columns = [col for col in df_train.columns if 'generation' in col]
df_train['generation_sum'] = df_train[generation_columns].sum(axis=1)
df_test['generation_sum'] = df_test[generation_columns].sum(axis=1)

In [91]:
# 発電量のうちの火力発電の構成比[fossil]再生可能エネルギー構成比[renewable]
df_train['generation_sum'] = df_train[generation_columns].sum(axis=1)
df_test['generation_sum'] = df_test[generation_columns].sum(axis=1)

# --- ② 分子になる電源グループ ---
df_train["fossil_total"]     = (
        df_train["generation_fossil_gas"]
      + df_train["generation_fossil_hard_coal"]
      + df_train["generation_fossil_oil"]
)
df_test["fossil_total"]      = (
        df_test["generation_fossil_gas"]
      + df_test["generation_fossil_hard_coal"]
      + df_test["generation_fossil_oil"]
)

df_train["renewable_total"]  = (
        df_train["generation_hydro_pumped_storage_consumption"]
      + df_train["generation_hydro_run_of_river_and_poundage"]
      + df_train["generation_hydro_water_reservoir"]
      + df_train["generation_other_renewable"]
)
df_test["renewable_total"]   = (
        df_test["generation_hydro_pumped_storage_consumption"]
      + df_test["generation_hydro_run_of_river_and_poundage"]
      + df_test["generation_hydro_water_reservoir"]
      + df_test["generation_other_renewable"]
)

# --- ③ 比率を安全に計算（0 除算防止） ---
eps = 1e-6
df_train["fossil_share"]     = df_train["fossil_total"]    / (df_train["generation_sum"] + eps)
df_test["fossil_share"]      = df_test["fossil_total"]     / (df_test["generation_sum"]  + eps)

df_train["renewable_share"]  = df_train["renewable_total"] / (df_train["generation_sum"] + eps)
df_test["renewable_share"]   = df_test["renewable_total"]  / (df_test["generation_sum"]  + eps)

In [92]:
# 祝日と3連休
import holidays, numpy as np

es_holidays = holidays.Spain(years=[2015, 2016, 2017, 2018])

for df in (df_train, df_test):
    df["is_holiday"] = df["time"].dt.date.map(lambda d: 1 if d in es_holidays else 0)
    # 前後1日
    for s in [-1, 1]:
        df[f"hol_adj{s}"] = df["is_holiday"].shift(s).fillna(0)
    # 3連休以上を flag
    df["is_long_wend"] = (
        (df["is_holiday"].rolling(3, min_periods=1).sum() >= 2).astype(int)
    )

In [93]:
# ---------------------------目的変数との相関が-0.24あったバレンシアの気温差---------------------------
# valencia_temp_diff
df_train["valencia_temp_diff"] = df_train["valencia_temp_max"] - df_train["valencia_temp_min"]
df_test["valencia_temp_diff"] = df_test["valencia_temp_max"] - df_test["valencia_temp_min"]

In [94]:
# ---------------------------チュートリアルにあった快適気温との差(valenciaのみ)---------------------------
# temp_dev
COMFORT = 23.0
df_train["temp_dev"] = (df_train["valencia_temp"] - COMFORT).abs()
df_test["temp_dev"] = (df_test["valencia_temp"] - COMFORT).abs()

# 不快指数


In [95]:
# ---------------------------交差項---------------------------
# season×is_weekend
df_train["season_isweekend"] = df_train["season"] * df_train["is_weekend"]
df_test["season_isweekend"] = df_test["season"] * df_test["is_weekend"]

# time_of_day×is_weekend
df_train["time_of_day_isweekend"] = df_train["time_of_day"] * df_train["is_weekend"]
df_test["time_of_day_isweekend"] = df_test["time_of_day"] * df_test["is_weekend"]

In [96]:
# --------------------- fossil_shareやrenewable_shareの変化量：風・太陽が落ちると火力比率↑ → 価格急騰。---------------------------------
# fossil_share_diff1
# renewable_share_diff1
# fossil_share_dev7d
# renewable_share_dev7d
share_cols = ["fossil_share", "renewable_share"]

for df in (df_train, df_test):
    # ---- ① 1 時間前との差分 ----
    for col in share_cols:
        df[f"{col}_diff1"] = df[col] - df[col].shift(1)

    # ---- ② 7日(168h) 移動平均からのずれ ----
    window = 24*7       # 168
    for col in share_cols:
        roll_mean = df[col].rolling(window, min_periods=1).mean()
        df[f"{col}_dev7d"] = df[col] - roll_mean

In [97]:
# ---------------------------price actualのラグ特徴量---------------------------
# ステップ 1：時系列順にソート
df_train = df_train.sort_values("time").reset_index(drop=True)
df_test = df_test.sort_values("time").reset_index(drop=True)

# ステップ 2：df_train にラグ追加（shift）
df_train["price_actual_lag1"]   = df_train["price_actual"].shift(1)
df_train["price_actual_lag24"]  = df_train["price_actual"].shift(24)
df_train["price_actual_lag168"] = df_train["price_actual"].shift(168)

# NaNを含む行を除外（df_train のみ）
df_train = df_train.dropna(subset=["price_actual_lag1", "price_actual_lag24", "price_actual_lag168"])

# ステップ 3：df_test にラグ追加（train の末尾を参照）
train_price = df_train.set_index("time")["price_actual"]

# 時間差からラグ生成（NaNがあっても dropna しない！）
for lag in [1, 24, 168]:
    df_test[f"price_actual_lag{lag}"] = df_test["time"].apply(
        lambda t: train_price.get(t - pd.Timedelta(hours=lag), np.nan)
    )

# ※ここでは df_test.dropna() しない → 提出用行数は保たれる

# モデルで使う test データだけ mask で抽出（あとで予測に使う）
test_mask = df_test[["price_actual_lag1", "price_actual_lag24", "price_actual_lag168"]].notna().all(axis=1)

# ✅ 表示確認（trainのみ）
display(df_train[["price_actual","price_actual_lag1","price_actual_lag24"]])

,price_actual,price_actual_lag1,price_actual_lag24
168,67.70,73.14,73.73
169,61.05,67.70,70.99
170,59.61,61.05,68.30
171,58.65,59.61,64.22
172,58.40,58.65,63.53
...,...,...,...
26275,39.90,35.37,62.92
26276,39.54,39.90,62.10
26277,32.90,39.54,60.09
26278,23.85,32.90,55.85


In [98]:
# ---------------------------totalload_actualのラグ特徴量---------------------------
lags = [1,24,168]
lag_targets = ["total_load_actual","generation_sum"]
for df in (df_train,df_test):
  for col in lag_targets:
    for i in lags:
      new_col = f"{col}_lag{i}"
      df[new_col] = df[col].shift(1)
      print(new_col)
df_train.head()

total_load_actual_lag1
total_load_actual_lag24
total_load_actual_lag168
generation_sum_lag1
generation_sum_lag24
generation_sum_lag168
total_load_actual_lag1
total_load_actual_lag24
total_load_actual_lag168
generation_sum_lag1
generation_sum_lag24
generation_sum_lag168


,time,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,...,renewable_share_dev7d,price_actual_lag1,price_actual_lag24,price_actual_lag168,total_load_actual_lag1,total_load_actual_lag24,total_load_actual_lag168,generation_sum_lag1,generation_sum_lag24,generation_sum_lag168
168,2015-01-08 23:00:00+00:00,546.0,571.0,4178.0,7280.0,383.0,398.0,658.0,831.0,6741.0,...,-0.104604,73.14,73.73,64.02,NaN,NaN,NaN,NaN,NaN,NaN
169,2015-01-09 00:00:00+00:00,516.0,566.0,3912.0,6774.0,370.0,392.0,628.0,942.0,6741.0,...,-0.098472,67.70,70.99,58.46,26788.0,26788.0,26788.0,28338.0,28338.0,28338.0
170,2015-01-09 01:00:00+00:00,508.0,455.0,3718.0,6349.0,372.0,956.0,631.0,882.0,6742.0,...,-0.077546,61.05,68.30,54.70,25146.0,25146.0,25146.0,27150.0,27150.0,27150.0
171,2015-01-09 02:00:00+00:00,509.0,369.0,3768.0,6078.0,373.0,1088.0,634.0,934.0,6743.0,...,-0.068615,59.61,64.22,54.91,23889.0,23889.0,23889.0,26577.0,26577.0,26577.0
172,2015-01-09 03:00:00+00:00,518.0,367.0,3707.0,5984.0,373.0,1027.0,636.0,799.0,6744.0,...,-0.073815,58.65,63.53,53.07,23046.0,23046.0,23046.0,26117.0,26117.0,26117.0


## データ分割＆モデル実装

### 特徴量定義

In [99]:
# 特徴量の整理

base_features = [
    'generation_fossil_gas',
    'generation_fossil_hard_coal',
    #'generation_fossil_oil',
    'generation_hydro_pumped_storage_consumption',
    'generation_hydro_run_of_river_and_poundage',
    'generation_hydro_water_reservoir',
    'generation_other_renewable',
    'total_load_actual',
]

add_features = [
    "generation_biomass",
    "generation_fossil_brown_coal/lignite",
    #"valencia_wind_speed",
    "madrid_wind_speed",
    "bilbao_pressure",
    "bilbao_wind_speed",
    #"bilbao_clouds_all",
    "barcelona_pressure",
    "barcelona_wind_speed",
    "seville_pressure",
    "seville_wind_deg",

    #"season",
    "time_of_day",
    "is_weekend",
    #"is_holiday",
    #"is_long_wend",
    #'generation_sum',
    #"fossil_share",
    #"renewable_share",
    #"time_of_day_only_night",
    "valencia_temp_diff",
    "season_isweekend",
    "time_of_day_isweekend",
    "temp_dev",
    "fossil_share_diff1",
    "renewable_share_diff1",
    "fossil_share_dev7d",
    "renewable_share_dev7d",
    "price_actual_lag1",
    #"price_actual_lag24",
    #"price_actual_lag168",
    "total_load_actual_lag1",
    "total_load_actual_lag24",
    #"total_load_actual_lag168",
    "generation_sum_lag1",
    #"generation_sum_lag24",
    #"generation_sum_lag168"
]

features = base_features + add_features

### パラメータ定義

In [100]:
params_xgb = {
    "objective":       "reg:squarederror",
    "learning_rate":   0.023378586570803068,
    "max_depth":       7,
    "n_estimators":    317,
    "subsample":       0.9665142871966073,
    "colsample_bytree":0.6519917349250606,
    "min_child_weight":1,
    "gamma":           0.02005153624696357,
    "lambda":          0.6361532102889825,  # L2 正則化
    "tree_method":     "hist",
    "random_state":    42,
    "eval_metric":     "rmse"
}

params_cb = {
    "iterations": 5000,
    "learning_rate": 0.05,
    "depth": 8,
    "loss_function": "RMSE",
    "eval_metric": "RMSE",
    "random_seed": 42,
    "od_type": "Iter",
    "od_wait": 200,   # early stopping
    "verbose": 200
}

params_lgb = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric':'rmse',
    'learning_rate':0.05,
    'num_leaves':12,
    'max_depth':-1,
    'n_estimators':5000,
    "force_col_wise": True,
    'importance_type':'gain'
}


"""



best_params_cb = {
    'iterations': 2855,
    'learning_rate': 0.09865989811944446,
    'depth': 4,
    'l2_leaf_reg': 0.00011419243952339203,
    "loss_function": "RMSE",
    "eval_metric": "RMSE",
    "random_seed": 42,
    "od_type": "Iter",
    "od_wait": 200,   # early stopping
    "verbose": 200
    }

best_params_lgb = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric':'rmse',
    'learning_rate': 0.019977381820598494,
    'num_leaves': 28,
    'feature_fraction': 0.9679195640722122,
    'bagging_fraction': 0.7093899186623671,
    'bagging_freq': 5,
    'lambda_l1': 0.0006846195699654229,
    'lambda_l2': 0.004414436544683086
    }
"""

'\n\n\n\nbest_params_cb = {\n    \'iterations\': 2855, \n    \'learning_rate\': 0.09865989811944446, \n    \'depth\': 4, \n    \'l2_leaf_reg\': 0.00011419243952339203,\n    "loss_function": "RMSE",\n    "eval_metric": "RMSE",\n    "random_seed": 42,\n    "od_type": "Iter",\n    "od_wait": 200,   # early stopping\n    "verbose": 200\n    }\n\nbest_params_lgb = {\n    \'boosting_type\':\'gbdt\',\n    \'objective\':\'regression\',\n    \'metric\':\'rmse\',\n    \'learning_rate\': 0.019977381820598494, \n    \'num_leaves\': 28, \n    \'feature_fraction\': 0.9679195640722122, \n    \'bagging_fraction\': 0.7093899186623671, \n    \'bagging_freq\': 5, \n    \'lambda_l1\': 0.0006846195699654229, \n    \'lambda_l2\': 0.004414436544683086\n    }\n'

### 各モデルの関数定義


In [101]:
# ------------------------------------------------------------
# XGBoost
# ------------------------------------------------------------
def run_xgb(df_train, train_mask, val_mask, features,tag,params_xgb):
    """複数の検証パターンでXGBoostを回すための関数
    Parameters
    ----------
    df_train : 学習用 DataFrame（特徴量列 + 'price_actual'）
    df_test   : 検証用 〃
    features: 学習に使う列名リスト
    tag      : ログに出力するラベル文字列
    params_xgb   : xgboost.XGBRegressor に渡すハイパーパラメータ辞書

    Returns
    -------
    model : 学習済み XGBRegressor
    rmse  : 検証 RMSE
    """
    X_tr = df_train.loc[train_mask, features]
    y_tr = df_train.loc[train_mask, "price_actual"]
    X_va = df_train.loc[val_mask,  features]
    y_va = df_train.loc[val_mask,  "price_actual"]

    model = xgb.XGBRegressor(**params_xgb)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        verbose=False,
    )

    pred_va = model.predict(X_va)
    mse = mean_squared_error(y_va, pred_va)
    rmse = np.sqrt(mse)
    print(f"[{tag}]  val RMSE = {rmse:.4f}")
    return model, rmse


In [102]:
# ------------------------------------------------------------
# CatBoost
# ------------------------------------------------------------

def run_cb(df_train, train_mask, val_mask, features,tag,params_cb):
    """複数の検証パターンでXGBoostを回すための関数
    Parameters
    ----------
    df_train : 学習用 DataFrame（特徴量列 + 'price_actual'）
    df_test   : 検証用 〃
    features: 学習に使う列名リスト
    tag      : ログに出力するラベル文字列
    params_xgb   : xgboost.XGBRegressor に渡すハイパーパラメータ辞書

    Returns
    -------
    model : 学習済み XGBRegressor
    rmse  : 検証 RMSE
    """
    X_tr = df_train.loc[train_mask, features]
    y_tr = df_train.loc[train_mask, "price_actual"]
    X_va = df_train.loc[val_mask,  features]
    y_va = df_train.loc[val_mask,  "price_actual"]

    train_pool = Pool(X_tr, y_tr)
    valid_pool = Pool(X_va, y_va)

    model_cb = CatBoostRegressor(**params_cb)
    model_cb.fit(
        train_pool,
        eval_set=valid_pool,
        use_best_model=True,
        verbose=False
        )

    pred_va = model_cb.predict(valid_pool)
    mse_cb = mean_squared_error(y_va, pred_va)
    rmse_cb = np.sqrt(mse_cb)

    print(f"[{tag}]  val RMSE = {rmse_cb:.4f}")
    return model_cb, rmse_cb

In [103]:
# ------------------------------------------------------------
# Light GBM
# ------------------------------------------------------------

def run_lgb(df_train, train_mask, val_mask, features,tag,params_lgb):
    """複数の検証パターンでXGBoostを回すための関数
    Parameters
    ----------
    df_train : 学習用 DataFrame（特徴量列 + 'price_actual'）
    df_test   : 検証用 〃
    features: 学習に使う列名リスト
    tag      : ログに出力するラベル文字列
    params_lgb   : modelに渡すハイパーパラメータ辞書

    Returns
    -------
    model : 学習済み
    rmse  : 検証 RMSE
    """
    X_tr = df_train.loc[train_mask, features]
    y_tr = df_train.loc[train_mask, "price_actual"]
    X_va = df_train.loc[val_mask,  features]
    y_va = df_train.loc[val_mask,  "price_actual"]

    model_lgb = lgb.LGBMRegressor(**params_lgb)
    model_lgb.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="rmse",
        callbacks=[lgb.early_stopping(100),
                    lgb.log_evaluation(50)]
    )

    pred_va = model_lgb.predict(X_va)
    mse = mean_squared_error(y_va, pred_va)
    rmse_lgb = np.sqrt(mse)
    print(f"[{tag}]  val RMSE = {rmse_lgb:.4f}")
    return model_lgb, rmse_lgb


In [115]:
"""
# ------------------------------------------------------------------
# 複数モデルの学習→結果、モデル3が良さそう6/19
# ------------------------------------------------------------------

# ------------------------------------------------------------------
# ① Train: 2015-01-01〜2016-12-31 / Val: 2017-01-01〜2017-12-31
# ------------------------------------------------------------------
mask_15_16 = df_train["time"].between("2015-01-01", "2016-12-31")
mask_17    = df_train["time"].between("2017-01-01", "2017-12-31")

model_1, rmse_1 = run_xgb(df_train,mask_15_16, mask_17, features, "Pattern-1_xb", params_xgb)

# ------------------------------------------------------------------
# ③ Train: 2015-01-01〜2017-09-30 / Val: 2017-10-01〜12-31
# ------------------------------------------------------------------
mask_tr_3 = df_train["time"].between("2015-01-01", "2017-09-30")
mask_va_3 = df_train["time"].between("2017-10-01", "2017-12-31")

model_3, rmse_3 = run_xgb(df_train,mask_tr_3, mask_va_3,features,"Pattern-3_xb",params_xgb)
model_cb_3, rmse_cb_3 = run_cb(df_train,mask_tr_3, mask_va_3,features,"Pattern-3_cb",params_cb)
model_lgb_3, rmse_lgb_3 = run_lgb(df_train,mask_tr_3, mask_va_3,features,"Pattern-3_lgb",params_lgb)
"""

# ------------------------------------------------------------------
# ④ Train: 2015-01-01〜2016-12-31（2015-08-01～2016-03-31を除く） / Val: 2017-01-01〜2017-12-31
# ------------------------------------------------------------------
# まず全期間（2015-01-01〜2016-12-31）のマスクを作成
mask_whole = df_train["time"].between("2015-01-01", "2016-12-31")

# カットしたい区間（2015-08-01〜2016-03-31）のマスク
mask_cut = df_train["time"].between("2015-08-01", "2016-03-31")

# “全期間” − “カット区間” ＝ 使いたい期間
mask_15_16 = mask_whole & ~mask_cut
mask_17    = df_train["time"].between("2017-01-01", "2017-12-31")

model_4, rmse_4 = run_xgb(df_train,mask_15_16, mask_17, features, "Pattern-4_xb", params_xgb)
model_cb_4, rmse_cb_4 = run_cb(df_train,mask_15_16, mask_17,features,"Pattern-4_cb",params_cb)
model_lgb_4, rmse_lgb_4 = run_lgb(df_train,mask_15_16, mask_17,features,"Pattern-4_lgb",params_lgb)

[Pattern-4_xb]  val RMSE = 3.9844
[Pattern-4_cb]  val RMSE = 3.5109
Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 4.00832
[100]	valid_0's rmse: 3.49365
[150]	valid_0's rmse: 3.38856
[200]	valid_0's rmse: 3.33928
[250]	valid_0's rmse: 3.32379
[300]	valid_0's rmse: 3.32286
[350]	valid_0's rmse: 3.31888
[400]	valid_0's rmse: 3.3155
[450]	valid_0's rmse: 3.31756
[500]	valid_0's rmse: 3.31493
[550]	valid_0's rmse: 3.31053
[600]	valid_0's rmse: 3.30252
[650]	valid_0's rmse: 3.29903
[700]	valid_0's rmse: 3.29961
Early stopping, best iteration is:
[623]	valid_0's rmse: 3.29673
[Pattern-4_lgb]  val RMSE = 3.2967


In [105]:
# ------------------------------------------------------------------
# バリデーション後、もったいないのですべてのデータをもって再学習 （2015-08-01～2016-03-31を除くVer）
# ------------------------------------------------------------------
mask_all =  (
    df_train["time"].between("2015-01-01", "2015-07-31") |
    df_train["time"].between("2016-04-01", "2016-12-31")
)
X_full = df_train.loc[mask_all, features]
y_full = df_train.loc[mask_all, "price_actual"]

model_full = xgb.XGBRegressor(**params_xgb)
model_full.fit(X_full, y_full)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6519917349250606, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=0.02005153624696357,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.6361532102889825,
             learning_rate=0.023378586570803068, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=317, n_jobs=None,
             num_parallel_tree=None, ...)

## ラグ逐次変化のトライ

In [37]:
# ------------------------------------------------------------
# 逐次予測 (recursive forecasting) ― price_actual_lag1 を安全利用
# ------------------------------------------------------------
# 前提：df_train / df_test / features / model_full が既に定義済み
#       features には "price_actual_lag1" を含めておく
#       （lag24 / lag168 を使うなら同様に列名を追加）

# 0) すべての特徴量を数値型へ統一（XGBoost が object を拒否するため）
for col in features:
    df_train[col] = pd.to_numeric(df_train[col], errors="coerce")
    df_test[col]  = pd.to_numeric(df_test[col],  errors="coerce")

# 1) 時系列順で test を並べ替え
df_test = df_test.sort_values("time").reset_index(drop=True)

# 2) ラグ用バッファを初期化
last_price = df_train.iloc[-1]["price_actual"]          # ← lag1 用
from collections import deque
buffer_24  = deque(df_train["price_actual"].tail(24).tolist(),  maxlen=24)
#buffer_168 = deque(df_train["price_actual"].tail(168).tolist(), maxlen=168)

# 3) 逐次ループで 1 行ずつ予測 → バッファ更新
pred_test = []

for _, row in df_test.iterrows():
    # --- ラグを上書き ---
    row_feats = row.copy()
    row_feats["price_actual_lag1"] = last_price
    row_feats["price_actual_lag24"]  = buffer_24[-1]   # ← 使うならアンコメント
    # row_feats["price_actual_lag168"] = buffer_168[-1]  # ← 使うならアンコメント

    # --- 1 行 DataFrame 化して数値型へ ---
    X_row = row_feats[features].astype(float).to_frame().T

    # --- 予測 ---
    y_hat = model_full.predict(X_row)[0]
    pred_test.append(y_hat)

    # --- バッファ更新 ---
    last_price = y_hat
    buffer_24.append(y_hat)
    #buffer_168.append(y_hat)

In [39]:
# 4) 提出ファイル作成

df_sub = pd.read_csv(file_path + 'sample_submit.csv',header=None)

submission = pd.DataFrame({
    'time':df_sub.iloc[:, 0] ,
    "pred": pred_test                 # 逐次予測した 8760 個
})
submission.to_csv("submission_recursive.csv", index=False, header=False)

## 加重平均のアンサンブル


In [77]:
# --- 1. 各モデルを「全データ」で再学習 ---
mask_all = (
    df_train["time"].between("2015-01-01", "2015-07-31") |
    df_train["time"].between("2016-04-01", "2016-12-31")
)
X_full = df_train.loc[mask_all, features]
y_full = df_train.loc[mask_all, "price_actual"]

# XGBoost
model_full_xgb = xgb.XGBRegressor(**params_xgb)
model_full_xgb.fit(X_full, y_full)

# LightGBM
model_full_lgb = lgb.LGBMRegressor(**params_lgb)
model_full_lgb.fit(X_full, y_full)

# CatBoost
model_full_cb = CatBoostRegressor(**params_cb)
model_full_cb.fit(X_full, y_full, verbose=False)

# --- 2. テストデータで予測 ---
X_test = df_test[features]  # 全8760行
pred_xgb = model_full_xgb.predict(X_test)
pred_lgb = model_full_lgb.predict(X_test)
pred_cb  = model_full_cb.predict(X_test)

# --- 3. バリデーションで得た RMSE を使って重み計算 ---
# (rmse_xgb, rmse_lgb, rmse_cb はバリデ結果から取得済み)
w_xgb = 1.0 / rmse_1
w_lgb = 1.0 / rmse_lgb_4
w_cb  = 1.0 / rmse_cb_4
w_sum = w_xgb + w_lgb + w_cb
w_xgb /= w_sum
w_lgb /= w_sum
w_cb  /= w_sum

print(f"weights -> XGB: {w_xgb:.3f}, LGB: {w_lgb:.3f}, CB: {w_cb:.3f}")

# --- 4. 重み付き平均アンサンブル ---
final_pred = pred_xgb * w_xgb + pred_lgb * w_lgb + pred_cb * w_cb

# --- 5. 提出用ファイル作成 ---
df_sub = pd.read_csv(file_path + 'sample_submit.csv', header=None)
submission = pd.DataFrame({
    "time": df_sub.iloc[:, 0],  # 8760行
    "pred": final_pred         # 8760長
})
submission.to_csv("ensemble_submission.csv", index=False, header=False)

KeyboardInterrupt: 

### 逐次予測のアンサンブル

In [117]:
# ------------------------------------------------------------------------------
# 前提：以下変数が定義済み
# df_train, df_test          # pandas.DataFrame
# features                   # 予測に使う特徴量リスト
# params_xgb, params_lgb, params_cb   # 各モデルのハイパーパラメータ dict
# rmse_4, rmse_lgb_4, rmse_cb_4       # Pattern-4 の検証で得た RMSE
# file_path                  # sample_submit.csv へのパス頭
# ------------------------------------------------------------------------------

# 1) 全データ再学習
mask_full = (
    df_train["time"].between("2015-01-01", "2015-07-31") |
    df_train["time"].between("2016-04-01", "2016-12-31")
)
X_full = df_train.loc[mask_full, features]
y_full = df_train.loc[mask_full, "price_actual"]

model_full_xgb = xgb.XGBRegressor(**params_xgb).fit(X_full, y_full, verbose=False)
model_full_lgb = lgb.LGBMRegressor(**params_lgb).fit(X_full, y_full)
model_full_cb  = CatBoostRegressor(**params_cb).fit(X_full, y_full, verbose=False)

# 2) 重み計算（逆数正規化）
w_xgb = 1.0 / rmse_4
w_lgb = 1.0 / rmse_lgb_4
w_cb  = 1.0 / rmse_cb_4
w_sum = w_xgb + w_lgb + w_cb
w_xgb, w_lgb, w_cb = w_xgb/w_sum, w_lgb/w_sum, w_cb/w_sum

# 3) テストを時間順にソート
df_test_rec = df_test.sort_values("time").reset_index(drop=True)

# 4) バッファ初期化
last_price = df_train["price_actual"].iloc[-1]
buf24  = deque(df_train["price_actual"].tail(24),  maxlen=24)
buf168 = deque(df_train["price_actual"].tail(168), maxlen=168)

# 5) 逐次予測＋アンサンブル
preds_recursive = []
for _, row in df_test_rec.iterrows():
    row_feats = row.copy()
    row_feats["price_actual_lag1"]   = last_price
    row_feats["price_actual_lag24"]  = buf24[-1]
    row_feats["price_actual_lag168"] = buf168[-1]

    X_row = row_feats[features].astype(float).to_frame().T
    y1 = model_full_xgb.predict(X_row)[0]
    y2 = model_full_lgb.predict(X_row)[0]
    y3 = model_full_cb.predict(X_row)[0]

    y_ens = w_xgb*y1 + w_lgb*y2 + w_cb*y3
    preds_recursive.append(y_ens)

    last_price = y_ens
    buf24.append(y_ens)
    buf168.append(y_ens)



In [114]:
# 6) 提出ファイル作成
df_sub = pd.read_csv(file_path + 'sample_submit.csv', header=None)
submission = pd.DataFrame({
    'time': df_sub.iloc[:, 0],
    'pred': preds_recursive    # 8760個の予測値
})
submission.to_csv("submission_recursive.csv", index=False, header=False)

出力行数: 8759


## 業務中に回すoptunaによる最適化

In [71]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# ——————————————————————————————————————————————————————
# ① マスク定義
# ——————————————————————————————————————————————————————
df_train["time"] = pd.to_datetime(df_train["time"])
mask_whole  = df_train["time"].between("2015-01-01", "2016-12-31")
mask_cut    = df_train["time"].between("2015-08-01", "2016-03-31")
train_mask  = mask_whole & ~mask_cut
val_mask    = df_train["time"].between("2017-01-01", "2017-12-31")

X_tr = df_train.loc[train_mask, features]
y_tr = df_train.loc[train_mask, "price_actual"]
X_va = df_train.loc[val_mask,   features]
y_va = df_train.loc[val_mask,   "price_actual"]


# ——————————————————————————————————————————————————————
# ② XGBoost 用 Optuna objective（低レベル API）
# ——————————————————————————————————————————————————————
def objective_xgb(trial):
    params = {
        "objective":    "reg:squarederror",
        "tree_method":  "hist",
        "eval_metric":  "rmse",
        "learning_rate":trial.suggest_loguniform("learning_rate",1e-3,1e-1),
        "max_depth":    trial.suggest_int("max_depth",4,12),
        "subsample":    trial.suggest_uniform("subsample",0.5,1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree",0.5,1.0),
        "gamma":        trial.suggest_loguniform("gamma",1e-8,1.0),
        "reg_lambda":   trial.suggest_loguniform("reg_lambda",1e-8,10.0),
        "reg_alpha":    trial.suggest_loguniform("reg_alpha",1e-8,10.0),
        "n_estimators": 1000,
        "verbosity":    0,
    }

    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_va, label=y_va)
    bst = xgb.train(
        params, dtrain,
        num_boost_round=params["n_estimators"],
        evals=[(dvalid, "validation")],
        early_stopping_rounds=50,
        verbose_eval=False,
    )

    # どちらか好きなほうを使ってください
    preds = bst.predict(dvalid, iteration_range=(0, bst.best_iteration))
    rmse = np.sqrt(mean_squared_error(y_va, preds))
    return rmse

# ——————————————————————————————————————————————————————
# ⑤ Study 実行
# ——————————————————————————————————————————————————————
study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_xgb, n_trials=50)
print("XGBoost best params:", study_xgb.best_params)

[I 2025-06-25 22:41:56,484] A new study created in memory with name: no-name-c700eb81-c3a9-4035-a71e-247c7ceed889
/tmp/ipython-input-71-2148788684.py:31: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate":trial.suggest_loguniform("learning_rate",1e-3,1e-1),
/tmp/ipython-input-71-2148788684.py:33: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample":    trial.suggest_uniform("subsample",0.5,1.0),
/tmp/ipython-input-71-2148788684.py:34: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "colsample_bytree": trial.suggest_uniform("colsample

KeyboardInterrupt: 

In [70]:

# ——————————————————————————————————————————————————————
# ③ LightGBM 用 Optuna objective（scikit API + callback）
# ——————————————————————————————————————————————————————
def objective_lgb(trial):
    params = {
        "objective":        "regression",
        "metric":           "rmse",
        "learning_rate":    trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "num_leaves":       trial.suggest_int("num_leaves", 8, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.5, 1.0),
        "bagging_freq":     trial.suggest_int("bagging_freq", 1, 10),
        "lambda_l1":        trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2":        trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "verbosity":        -1,
    }

    # ※ params_lgb → params に修正、n_estimators を明示
    model_lgb = lgb.LGBMRegressor(n_estimators=1000, **params)
    model_lgb.fit(
        X_tr,
        y_tr,
        eval_set=[(X_va, y_va)],
        callbacks=[
            early_stopping(stopping_rounds=50),
            log_evaluation(period=50),
        ],
    )

    preds = model_lgb.predict(X_va, num_iteration=model_lgb.best_iteration_)
    rmse = np.sqrt(mean_squared_error(y_va, preds))
    return rmse


study_lgb = optuna.create_study(direction="minimize")
study_lgb.optimize(objective_lgb, n_trials=50)
print("LightGBM best params:", study_lgb.best_params)


[I 2025-06-25 22:28:39,055] A new study created in memory with name: no-name-d9d7699d-58f7-438c-ba1a-3ea698a67d32
/tmp/ipython-input-70-1254361168.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate":    trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
/tmp/ipython-input-70-1254361168.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "feature_fraction": trial.suggest_uniform("feature_fraction", 0.5, 1.0),
/tmp/ipython-input-70-1254361168.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "bagging_fraction": trial.suggest_

Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 8.7254
[100]	valid_0's rmse: 6.24804
[150]	valid_0's rmse: 4.89998
[200]	valid_0's rmse: 4.23037
[250]	valid_0's rmse: 3.90061
[300]	valid_0's rmse: 3.71862
[350]	valid_0's rmse: 3.60869
[400]	valid_0's rmse: 3.55585
[450]	valid_0's rmse: 3.53651
[500]	valid_0's rmse: 3.52573
[550]	valid_0's rmse: 3.51498
Early stopping, best iteration is:
[524]	valid_0's rmse: 3.51428


[I 2025-06-25 22:28:49,361] Trial 0 finished with value: 3.514277410113399 and parameters: {'learning_rate': 0.01051148269318115, 'num_leaves': 93, 'feature_fraction': 0.8426668631026613, 'bagging_fraction': 0.5351828125962763, 'bagging_freq': 9, 'lambda_l1': 6.801134531148868e-07, 'lambda_l2': 0.10605179238252282}. Best is trial 0 with value: 3.514277410113399.
/tmp/ipython-input-70-1254361168.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate":    trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
/tmp/ipython-input-70-1254361168.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "feature_fraction": trial.suggest_uniform("feature_fraction", 0.5, 1.0),
/tmp/ipython

Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 9.89782
[100]	valid_0's rmse: 7.93159
[150]	valid_0's rmse: 6.56026
[200]	valid_0's rmse: 5.69557
[250]	valid_0's rmse: 5.19011
[300]	valid_0's rmse: 4.84869
[350]	valid_0's rmse: 4.62206
[400]	valid_0's rmse: 4.47355
[450]	valid_0's rmse: 4.37464
[500]	valid_0's rmse: 4.3169
[550]	valid_0's rmse: 4.26857
[600]	valid_0's rmse: 4.22134
[650]	valid_0's rmse: 4.18886
[700]	valid_0's rmse: 4.16925
[750]	valid_0's rmse: 4.15628
[800]	valid_0's rmse: 4.14186
[850]	valid_0's rmse: 4.13662
[900]	valid_0's rmse: 4.12758
[950]	valid_0's rmse: 4.11822
[1000]	valid_0's rmse: 4.11598
Did not meet early stopping. Best iteration is:
[991]	valid_0's rmse: 4.11561


[I 2025-06-25 22:29:02,660] Trial 1 finished with value: 4.115608757474285 and parameters: {'learning_rate': 0.009276630547840364, 'num_leaves': 83, 'feature_fraction': 0.5308656013731461, 'bagging_fraction': 0.5524803312385136, 'bagging_freq': 5, 'lambda_l1': 4.729244729873902e-07, 'lambda_l2': 0.10406217916998052}. Best is trial 0 with value: 3.514277410113399.
/tmp/ipython-input-70-1254361168.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate":    trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
/tmp/ipython-input-70-1254361168.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "feature_fraction": trial.suggest_uniform("feature_fraction", 0.5, 1.0),
/tmp/ipytho

...

Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 6.43225
[100]	valid_0's rmse: 4.23845
[150]	valid_0's rmse: 3.62041
[200]	valid_0's rmse: 3.43128
[250]	valid_0's rmse: 3.37692
[300]	valid_0's rmse: 3.35729
[350]	valid_0's rmse: 3.34327
[400]	valid_0's rmse: 3.35054
Early stopping, best iteration is:
[365]	valid_0's rmse: 3.33964


[I 2025-06-25 22:40:48,411] Trial 48 finished with value: 3.3396387468889275 and parameters: {'learning_rate': 0.01838929529579849, 'num_leaves': 86, 'feature_fraction': 0.975735769891036, 'bagging_fraction': 0.6977997862415386, 'bagging_freq': 5, 'lambda_l1': 0.00755063914210866, 'lambda_l2': 0.00436594410677167}. Best is trial 47 with value: 3.241045505147626.
/tmp/ipython-input-70-1254361168.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate":    trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
/tmp/ipython-input-70-1254361168.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "feature_fraction": trial.suggest_uniform("feature_fraction", 0.5, 1.0),
/tmp/ipython

Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 6.33952
[100]	valid_0's rmse: 4.24341
[150]	valid_0's rmse: 3.6627
[200]	valid_0's rmse: 3.45072
[250]	valid_0's rmse: 3.37351
[300]	valid_0's rmse: 3.32468
[350]	valid_0's rmse: 3.30406
[400]	valid_0's rmse: 3.29049
[450]	valid_0's rmse: 3.2783
[500]	valid_0's rmse: 3.28263
Early stopping, best iteration is:
[465]	valid_0's rmse: 3.27671


[I 2025-06-25 22:40:51,887] Trial 49 finished with value: 3.2767149730005243 and parameters: {'learning_rate': 0.019713092409206018, 'num_leaves': 22, 'feature_fraction': 0.9727406466843518, 'bagging_fraction': 0.689483940441268, 'bagging_freq': 5, 'lambda_l1': 0.00865886226711966, 'lambda_l2': 6.503249700311134e-05}. Best is trial 47 with value: 3.241045505147626.


LightGBM best params: {'learning_rate': 0.019977381820598494, 'num_leaves': 28, 'feature_fraction': 0.9679195640722122, 'bagging_fraction': 0.7093899186623671, 'bagging_freq': 5, 'lambda_l1': 0.0006846195699654229, 'lambda_l2': 0.004414436544683086}


In [69]:

# ——————————————————————————————————————————————————————
# ④ CatBoost 用 Optuna objective（変更なしで OK）
# ——————————————————————————————————————————————————————
def objective_cb(trial):
    params = {
        "iterations":    trial.suggest_int("iterations", 1000, 3000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "depth":         trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg":   trial.suggest_loguniform("l2_leaf_reg", 1e-8, 10.0),
        "random_seed":   42,
        "loss_function": "RMSE",
        "verbose":       False,
    }
    model = CatBoostRegressor(**params)
    model.fit(
        X_tr,
        y_tr,
        eval_set=(X_va, y_va),
        early_stopping_rounds=50,
        verbose=False,
    )
    preds = model.predict(X_va)
    rmse = np.sqrt(mean_squared_error(y_va, preds))
    return rmse


study_cb = optuna.create_study(direction="minimize")
study_cb.optimize(objective_cb, n_trials=50)
print("CatBoost best params:", study_cb.best_params)

[I 2025-06-25 22:18:33,104] A new study created in memory with name: no-name-9b506b4c-496e-4b11-b3b4-539c4fe0fcb3
/tmp/ipython-input-69-162667526.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
/tmp/ipython-input-69-162667526.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "l2_leaf_reg":   trial.suggest_loguniform("l2_leaf_reg", 1e-8, 10.0),
[I 2025-06-25 22:19:07,696] Trial 0 finished with value: 3.495419763508533 and parameters: {'iterations': 2113, 'learning_rate': 0.013941445369000532, 'depth': 8, 'l2_leaf_reg': 0.000768143067958579}. Best is trial 0 with value: 3.495419763508533.
/tmp/i

CatBoost best params: {'iterations': 2855, 'learning_rate': 0.09865989811944446, 'depth': 4, 'l2_leaf_reg': 0.00011419243952339203}
